# pyqt

## Installation

    pip install pyqt5

## Erstellen der GUI

In [7]:
ls mainwindow.ui

mainwindow.ui


In [29]:
! qtcreator mainwindow.ui

In [15]:
! pyuic5 -x -o mainwindow.py mainwindow.ui

In [16]:
!cat mainwindow.py

# -*- coding: utf-8 -*-

# Form implementation generated from reading ui file 'mainwindow.ui'
#
# Created by: PyQt5 UI code generator 5.11.3
#
# WARNING! All changes made in this file will be lost!

from PyQt5 import QtCore, QtGui, QtWidgets

class Ui_mainWindow(object):
    def setupUi(self, mainWindow):
        mainWindow.setObjectName("mainWindow")
        mainWindow.resize(475, 381)
        self.centralWidget = QtWidgets.QWidget(mainWindow)
        self.centralWidget.setObjectName("centralWidget")
        self.btn_submit = QtWidgets.QPushButton(self.centralWidget)
        self.btn_submit.setGeometry(QtCore.QRect(10, 300, 85, 27))
        self.btn_submit.setObjectName("btn_submit")
        self.tableView = QtWidgets.QTableView(self.centralWidget)
        self.tableView.setGeometry(QtCore.QRect(0, 0, 471, 291))
        self.tableView.setSortingEnabled(True)
        self.tableView.setObjectName("tableView")
        self.lbl_status = QtWidgets.QLabel(self.centralWidget)
        self.lb

## Hauptprogramm

## Erstellen einer Datenbank

In [9]:
! rm db.sqlite

In [11]:
DB_NAME = 'db.sqlite'
NUM_INIT_DATA = 10
SQL_CREATE = '''
    CREATE TABLE IF NOT EXISTS person 
    (
    id integer primary key autoincrement,
    name text not null,
    geschlecht text not null,
    gebjahr integer not null,
    CONSTRAINT check_correct_val CHECK (geschlecht in ('w', 'm', 'x'))
    )
'''

Beispieldaten in die Datenbank einfügen

In [13]:
import sqlite3
import random

conn = sqlite3.connect(DB_NAME)
c = conn.cursor()
c.execute(SQL_CREATE)

for id in range(NUM_INIT_DATA):
    gebjahr = random.randint(1950, 2010)
    name = random.choice(['Peter', 'Petra', 'Susi', 'Moni'])
    geschlecht = random.choice(['m', 'w', 'x'])

    c.execute('''INSERT INTO person (id,name,geschlecht,gebjahr) 
                 VALUES (?,?,?,?)''',
              (id, name, geschlecht, gebjahr))

conn.commit()
conn.close()

Mit sqlite3 können die Daten bereits auf der Kommandozeile ausgelesen werden.

In [17]:
! sqlite3 --column --header db.sqlite 'SELECT * FROM person'

id          name        geschlecht  gebjahr   
----------  ----------  ----------  ----------
0           Moni        w           1956      
1           Petra       w           1972      
2           Moni        m           1956      
3           Susi        m           1959      
4           Petra       x           1994      
5           Peter       w           1986      
6           Petra       m           1960      
7           Petra       m           2005      
8           Petra       m           1985      
9           Peter       m           1981      


In [27]:
import sys
import random
import sqlite3

import mainwindow
from PyQt5.QtWidgets import QApplication, QDialog, QMainWindow
from PyQt5.QtSql import QSqlTableModel, QSqlDatabase

class MainWindow(mainwindow.Ui_mainWindow):
    def setupUi(self, mainwindow):
        super().setupUi(mainwindow)
        self.btn_submit.clicked.connect(self.submit_clicked)
        self.btn_add_row.clicked.connect(self.add_row_clicked)

        db = QSqlDatabase.addDatabase('QSQLITE')
        db.setDatabaseName(DB_NAME)

        self.tablemodel = QSqlTableModel()
        self.tablemodel.setTable('person')
        self.tablemodel.setEditStrategy(QSqlTableModel.OnManualSubmit)
        self.tablemodel.select()  # populate model with data

        self.tableView.setModel(self.tablemodel)
        self.tableView.hideColumn(0)  # hide ids

    def submit_clicked(self):
        succ = self.tablemodel.submitAll()
        self.lbl_status.setText("Daten erfolgreich gespeichert? %s" % succ)

    def add_row_clicked(self):
        self.tablemodel.insertRows(self.tablemodel.rowCount(), 1)

def initdb():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS person 
        (
        id integer primary key autoincrement,
        name text not null,
        geschlecht text not null,
        gebjahr integer not null,
        CONSTRAINT check_correct_val CHECK (geschlecht in ('w', 'm', 'x'))
        )
        ''')

    rows = c.execute('SELECT * FROM person').fetchall()
    if len(rows) > 0:
        print(len(rows), "data entries found")
    else:
        # init with sample data
        for id in range(NUM_INIT_DATA):
            gebjahr = random.randint(1950, 2010)
            name = random.choice(['Peter', 'Petra', 'Susi', 'Moni'])
            geschlecht = random.choice(['m', 'w', 'x'])

            c.execute('''INSERT INTO person (id,name,geschlecht,gebjahr) 
                         VALUES (?,?,?,?)''',
                      (id, name, geschlecht, gebjahr))

    conn.commit()
    conn.close()

def main():
    initdb()

    app = QApplication(sys.argv)
    window = QMainWindow()
    ui = MainWindow()
    ui.setupUi(window)
    window.show()

    sys.exit(app.exec_())

if __name__ == '__main__':
    main()


SystemExit: 0

/home/marco/proj/software-101/venv/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
